# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import glob
import geometric_sampling as gs
import pandas as pd
import numpy as np
import itertools
from tqdm import tqdm
import rpy2.robjects as ro
from rpy2.robjects import numpy2ri, default_converter
from rpy2.robjects.conversion import localconverter
from package_sampling.utils import inclusion_probabilities

/Users/mehdi/Documents/Projects/graphical_sampling/.venv/lib/python3.10/site-packages/rpy2/rinterface/__init__.py:1211: UserWarning: Environment variable "PWD" redefined by R and overriding existing variable. Current: "/Users/mehdi/Documents/Projects/graphical_sampling", R: "/Users/mehdi/Documents/Projects/graphical_sampling/Simulations"
  warnings.warn(
/Users/mehdi/Documents/Projects/graphical_sampling/.venv/lib/python3.10/site-packages/rpy2/rinterface/__init__.py:1211: UserWarning: Environment variable "XPC_SERVICE_NAME" redefined by R and overriding existing variable. Current: "application.com.jetbrains.pycharm.1003326.1004450", R: "0"
  warnings.warn(
/Users/mehdi/Documents/Projects/graphical_sampling/.venv/lib/python3.10/site-packages/rpy2/rinterface/__init__.py:1211: UserWarning: Environment variable "R_SESSION_TMPDIR" redefined by R and overriding existing variable. Current: "/var/folders/tw/x1njhbdj7q19p65g47mw5dqm0000gn/T//RtmpOBxN13", R: "/var/folders/tw/x1njhbdj7q19p65g47mw

In [3]:
%load_ext rpy2.ipython

In [4]:
%%R

library(WaveSampling)
library(sampling)
library(BalancedSampling)


Loading required package: Matrix


# Moran and Local Balance Score

In [5]:
def score_all_samples_moran_lb(coords, probs, sample_indices_list):
    """
    coords          : an (N×2)-array of spatial coordinates
    probs           : length-N array of inclusion probabilities
    sample_indices_list : list of length-n integer numpy arrays (0-based indices)

    Returns an (S×2) numpy array of [IB, SBLB] for each of the S samples.
    """

    # Convert Python list of numpy arrays into an R list of integer vectors
    #   * add +1 because R is 1-based
    r_sample_list = ro.ListVector({
        str(i+1): ro.IntVector(sample_idx.astype(int) + 1)
        for i, sample_idx in enumerate(sample_indices_list)
    })

    with localconverter(default_converter + numpy2ri.converter):
        ro.globalenv['coords'] = coords
        ro.globalenv['probs'] = probs
        # Precompute W once
        ro.r("""
            W0 <- wpik(coords, probs)
            W <- W0 - diag(diag(W0))
            diag(W) <- 0
        """)
        ro.globalenv['samples'] = r_sample_list

        # Define an R function that loops over all samples
        ro.r("""
            score_samples <- function(W, probs, coords, samples_list) {
              S <- length(samples_list)
              IBs   <- numeric(S)
              SBLBs <- numeric(S)

              for (i in seq_len(S)) {
                samp_idx <- samples_list[[i]]
                mask <- integer(length(probs))
                mask[samp_idx] <- 1

                IBs[i]   <- tryCatch(IB(W, mask),        error = function(e) Inf)
                SBLBs[i] <- tryCatch(sblb(probs, coords, samp_idx), error = function(e) Inf)
              }
              # return as a 2-column matrix
              cbind(IB = IBs, SBLB = SBLBs)
            }
        """)

        # Call it once
        result = ro.r("score_samples(W, probs, coords, samples)")
        # result comes back as an R matrix  S×2

    # Turn it into an (S×2) numpy array
    with localconverter(default_converter + numpy2ri.converter):
        np_result = np.array(result)
    return np_result[:, 0], np_result[:, 1]


# Loading Population

In [6]:
DATA_DIR = "../data_samples/coords_probs"
csv_paths = glob.glob(os.path.join(DATA_DIR, "*.csv"))

coords_dict = {}
probs_dict = {}

for fp in csv_paths:
    if 'swiss' not in fp:
        name = os.path.splitext(os.path.basename(fp))[0]
        data = np.loadtxt(fp, delimiter=",", skiprows=1)
        coords = data[:, :2]
        probs  = data[:, -1]

        coord_name, prob_name = name.split("_")
        coord_name = 'cluster' if coord_name == 'clust' else coord_name
        prob_name = 'equal' if prob_name == 'eq' else 'unequal'

        coords_dict[coord_name] = coords
        probs_dict[coord_name] = probs_dict.get(coord_name, {})
        probs_dict[coord_name][prob_name] = probs

print(coords_dict.keys())
print(probs_dict.keys())
print(probs_dict['random'].keys())

dict_keys(['random', 'cluster', 'meuse', 'grid'])
dict_keys(['random', 'cluster', 'meuse', 'grid'])
dict_keys(['equal', 'unequal'])


# Evaluation Function

In [7]:
def top_n_records(df: pd.DataFrame,
                  name_cols: list,
                  sort_col: str = 'exp_moran',
                  k: int = 10,
                  smallest: bool = True) -> pd.DataFrame:

    df_copy = df.copy()

    df_copy['name'] = df_copy[name_cols].astype(str).agg(' - '.join, axis=1)

    if smallest:
        return df_copy.nsmallest(k, sort_col)[['name', sort_col]].reset_index(drop=True)
    return df_copy.nlargest(k, sort_col)[['name', sort_col]].reset_index(drop=True)

In [45]:
def evaluate(
        coords_dict,
        probs_dict,
        n_values=[4, 8, 16],
        zone_list=[(1, 1), (2, 2), (3, 3)],
        sort_method_list=['lexico', 'random', 'angle_0', 'distance_0', 'projection', "center", "spiral", "max"],
        zonal_sort_list=['lexico', 'random', 'angle_0', 'distance_0', 'projection', "center", "spiral", "max"],
        tolerance=5,
        split_size=1e-3,
):
    records = []

    # pre‐compute all combinations
    combos = list(itertools.product(
        n_values,
        zone_list,
        sort_method_list,
        zonal_sort_list
    ))

    for n, zones, sort_method, zonal_sort in tqdm(
        combos,
        desc="Total combos",
        unit="combo"
    ):

        for coord_name in probs_dict.keys():
            for prob_name in probs_dict[coord_name].keys():

                coords = coords_dict[coord_name]
                probs = probs_dict[coord_name][prob_name]

                kss = gs.sampling.KMeansSpatialSamplingSimple(
                    coords, probs,
                    n=n,
                    n_zones=zones,
                    tolerance=tolerance,
                    split_size=split_size,
                    sort_method = sort_method,
                    zonal_sort = zonal_sort,
                    zone_mode='cluster'
                )

                density_expected = np.round(kss.expected_score(), 4)
                density_val = np.round(kss.var_score(), 4)

                moran_scores, lb_scores = score_all_samples_moran_lb(coords, probs, kss.all_samples)

                moran_expected = np.round(kss.expected_score(moran_scores), 4)
                moran_val = np.round(kss.var_score(moran_scores), 4)

                lb_expected = np.round(kss.expected_score(lb_scores), 4)
                lb_val = np.round(kss.var_score(lb_scores), 4)

                records.append({
                    # 'n': n,
                    # 'zones': f"{zones[0]}×{zones[1]}",
                    'zones': zones,
                    'bar_sort': sort_method,
                    'zonal_sort': zonal_sort if zonal_sort else 'None',
                    'coord': coord_name,
                    'prob': prob_name,
                    'exp_density': density_expected,
                    'exp_moran': moran_expected,
                    'exp_lb': lb_expected,
                    'var_density': density_val,
                    'var_moran': moran_val,
                    'var_lb': lb_val,
                })

    return pd.DataFrame.from_records(records)


In [48]:
df = evaluate(
    coords_dict,
    probs_dict,
    n_values=[4],
    # zone_list=[(1, 1), (2, 2), (3, 3)],
    zone_list=[2, 3, 4],
    sort_method_list=['distance_0', 'projection'],
    zonal_sort_list=[None, 'distance_0', 'projection'],
    tolerance=5,
    split_size=1e-3,
)

Total combos: 100%|██████████| 18/18 [04:34<00:00, 15.26s/combo]


In [49]:
summary = df.pivot_table(
    columns=['zones', 'zonal_sort', 'bar_sort'],
    # values=['exp_density', 'exp_moran', 'exp_lb', 'var_density', 'var_moran', 'var_lb'],
    values=['exp_moran',],
    index=['coord', 'prob'],
    aggfunc='first'
)

summary

exp_moran                                              \
zones                    2                                               
zonal_sort            None            distance_0            projection   
bar_sort        distance_0 projection distance_0 projection distance_0   
coord   prob                                                             
cluster equal      -0.1253    -0.1276    -0.1985    -0.2043    -0.1915   
        unequal    -0.0786    -0.1228    -0.1248    -0.1248    -0.1248   
grid    equal      -0.1116    -0.2019    -0.1781    -0.1758    -0.1758   
        unequal    -0.1174    -0.0574    -0.1174    -0.1174    -0.1174   
meuse   equal      -0.2683    -0.2547    -0.2745    -0.2745    -0.2745   
        unequal    -0.1968    -0.1676    -0.2193    -0.2183    -0.2193   
random  equal      -0.1618    -0.1318    -0.1603    -0.1618    -0.1600   
        unequal    -0.0869    -0.1104    -0.1336    -0.1341    -0.1336   

                                                                        \
zones                               3                                    
zonal_sort                       None            distance_0              
bar_sort        projection distance_0 projection distance_0 projection   
coord   prob                                                             
cluster equal      -0.1985    -0.2449    -0.1455    -0.2392    -0.2384   
        unequal    -0.1248    -0.1149    -0.1299    -0.1058    -0.1294   
grid    equal      -0.1781    -0.1489    -0.1537    -0.1900    -0.1576   
        unequal    -0.1174    -0.0971    -0.0973    -0.1215    -0.1215   
meuse   equal      -0.2745    -0.2612    -0.1856    -0.3335    -0.3335   
        unequal    -0.2183    -0.1839    -0.1416    -0.2228    -0.2228   
random  equal      -0.1603    -0.1551    -0.1615    -0.1573    -0.1573   
        unequal    -0.1336    -0.0977    -0.1498    -0.1219    -0.1274   

                                                                        \
zones                                          4                         
zonal_sort      projection                  None            distance_0   
bar_sort        distance_0 projection distance_0 projection distance_0   
coord   prob                                                             
cluster equal      -0.2385    -0.2485    -0.3012    -0.2648    -0.3162   
        unequal    -0.1068    -0.1083    -0.1405    -0.0956    -0.1454   
grid    equal      -0.1708    -0.1478    -0.1527    -0.1396    -0.2041   
        unequal    -0.1229    -0.1197    -0.0938    -0.1060    -0.1224   
meuse   equal      -0.3335    -0.3335    -0.2753    -0.1998    -0.2930   
        unequal    -0.2228    -0.2228    -0.2206    -0.1668    -0.2395   
random  equal      -0.1571    -0.1568    -0.1502    -0.1106    -0.1756   
        unequal    -0.1245    -0.1245    -0.1096    -0.0926    -0.1158   

                                                  
zones                                             
zonal_sort                 projection             
bar_sort        projection distance_0 projection  
coord   prob                                      
cluster equal      -0.3118    -0.3211    -0.3240  
        unequal    -0.1450    -0.1302    -0.1267  
grid    equal      -0.1683    -0.1817    -0.1761  
        unequal    -0.1186    -0.1085    -0.1220  
meuse   equal      -0.2926    -0.3298    -0.3271  
        unequal    -0.2307    -0.2223    -0.2203  
random  equal      -0.1734    -0.1756    -0.1756  
        unequal    -0.1158    -0.1112    -0.1116

# Meuse

In [25]:
def evaluate_meuse(
        coords_dict,
        probs_dict,
        n_values=[4],
        zone_list=[(1, 1), (2, 2), (3, 3)],
        sort_method_list=['lexico', 'random', 'angle_0', 'distance_0', 'projection', "center", "spiral", "max"],
        zonal_sort_list=['lexico', 'random', 'angle_0', 'distance_0', 'projection', "center", "spiral", "max"],
        zone_mode_list=['sweep' 'cluster'],
        tolerance=5,
        split_size=1e-3,
):
    records = []

    # pre‐compute all combinations
    combos = list(itertools.product(
        n_values,
        zone_list,
        zone_mode_list,
        zonal_sort_list,
        sort_method_list
    ))
    coords = coords_dict['meuse']
    probs = probs_dict['meuse']['equal']

    best_moran_sofar = 0

    for n, zones, zone_mode, zonal_sort, sort_method in tqdm(
        combos,
        desc="Total combos",
        unit="combo"
    ):

        # print(n, zones, zone_mode, zonal_sort, sort_method)

        modified_probs = inclusion_probabilities(probs, n=n)
        kss = gs.sampling.KMeansSpatialSamplingSimple(
            coords, modified_probs,
            n=n,
            n_zones=zones,
            tolerance=tolerance,
            split_size=split_size,
            zone_mode=zone_mode,
            sort_method=sort_method,
            zonal_sort=zonal_sort,
            max_missed_samples=1
        )

        density_expected = np.round(kss.expected_score(), 4)
        density_val = np.round(kss.var_score(), 4)

        moran_scores, lb_scores = score_all_samples_moran_lb(coords, modified_probs, kss.all_samples)

        moran_expected = np.round(kss.expected_score(moran_scores), 4)
        moran_val = np.round(kss.var_score(moran_scores), 4)

        lb_expected = np.round(kss.expected_score(lb_scores), 4)
        lb_val = np.round(kss.var_score(lb_scores), 4)

        if moran_expected < best_moran_sofar:
            best_moran_sofar = moran_expected
            print()

        records.append({
            'n': n,
            'zones': zones if zone_mode == 'cluster' else f"{zones[0]}×{zones[1]}",
            'zone_mode': zone_mode,
            'bar_sort': sort_method,
            'zonal_sort': zonal_sort if zonal_sort else 'None',
            'exp_density': density_expected,
            'exp_moran': moran_expected,
            'exp_lb': lb_expected,
            'var_density': density_val,
            'var_moran': moran_val,
            'var_lb': lb_val,
        })

    return pd.DataFrame.from_records(records)


In [26]:
df = evaluate_meuse(
    coords_dict,
    probs_dict,
    n_values=[15],
    # zone_list=[(1, 1), (2, 2)],
    zone_list=[1, 2, 3, 4, 5],
    sort_method_list=['lexico', 'random', 'distance_0', 'projection', "max"],
    zonal_sort_list=[None,'lexico', 'random', 'distance_0', 'projection', "max"],
    # zonal_sort_list=[None],
    zone_mode_list=['cluster'],
    tolerance=5,
    split_size=1e-3,
)

Total combos:   1%|          | 1/150 [00:01<04:34,  1.84s/combo]

Move border units INSIDE zones...
3 of them have been solved.
Current number of missed samples: 0
Lost Probs: 0.0


Total combos:   1%|▏         | 2/150 [00:03<04:08,  1.68s/combo]

Move border units INSIDE zones...
1 of them have been solved.
Current number of missed samples: 0
Lost Probs: 0.0


Total combos:   2%|▏         | 3/150 [00:05<04:17,  1.75s/combo]

...

Lost Probs: -2.220446049250313e-16


Total combos:  99%|█████████▉| 149/150 [06:24<00:02,  2.72s/combo]

Lost Probs: -2.220446049250313e-16


Total combos: 100%|██████████| 150/150 [06:27<00:00,  2.58s/combo]

Move border units INSIDE zones...
3 of them have been solved.
Current number of missed samples: 0
Lost Probs: 0.0


In [27]:
summary = df.pivot_table(
    values=['exp_density', 'exp_moran', 'exp_lb'],
    columns=['n', 'zones'],
    index=['zonal_sort', 'bar_sort'],
    aggfunc='first'
)

summary

exp_density                                  exp_lb  \
n                              15                                      15   
zones                           1       2       3       4       5       1   
zonal_sort bar_sort                                                         
None       distance_0      0.0221  0.0293  0.0490  0.0306  0.0578  0.1651   
           lexico          0.0418  0.0416  0.0469  0.0420  0.0305  0.1794   
           max             0.0301  0.0330  0.0398  0.0448  0.0522  0.1850   
           projection      0.0328  0.0461  0.0464  0.0421  0.0400  0.1811   
           random          0.0447  0.0570  0.0455  0.0426  0.0357  0.1735   
distance_0 distance_0      0.0304  0.0222  0.0337  0.0334  0.0330  0.1758   
           lexico          0.0384  0.0453  0.0366  0.0372  0.0336  0.1750   
           max             0.0280  0.0329  0.0230  0.0297  0.0359  0.1860   
           projection      0.0225  0.0268  0.0274  0.0330  0.0430  0.1861   
           random          0.0392  0.0428  0.0378  0.0363  0.0385  0.1715   
lexico     distance_0      0.0275  0.0297  0.0428  0.0425  0.0425  0.1687   
           lexico          0.0436  0.0396  0.0502  0.0431  0.0336  0.1802   
           max             0.0366  0.0424  0.0373  0.0315  0.0436  0.1664   
           projection      0.0225  0.0409  0.0450  0.0244  0.0432  0.1869   
           random          0.0535  0.0417  0.0386  0.0392  0.0558  0.1793   
max        distance_0      0.0319  0.0240  0.0384  0.0471  0.0314  0.1802   
           lexico          0.0347  0.0352  0.0302  0.0320  0.0305  0.1729   
           max             0.0343  0.0286  0.0325  0.0356  0.0341  0.1666   
           projection      0.0270  0.0330  0.0312  0.0241  0.0373  0.1827   
           random          0.0379  0.0236  0.0375  0.0314  0.0362  0.1719   
projection distance_0      0.0273  0.0302  0.0318  0.0268  0.0293  0.1843   
           lexico          0.0347  0.0383  0.0278  0.0320  0.0317  0.1798   
           max             0.0382  0.0290  0.0326  0.0309  0.0323  0.1779   
           projection      0.0318  0.0334  0.0285  0.0323  0.0386  0.1786   
           random          0.0424  0.0310  0.0391  0.0328  0.0294  0.1764   
random     distance_0      0.0333  0.0431  0.0430  0.0310  0.0511  0.1817   
           lexico          0.0397  0.0364  0.0451  0.0320  0.0350  0.1782   
           max             0.0324  0.0322  0.0429  0.0428  0.0403  0.1765   
           projection      0.0271  0.0531  0.0418  0.0493  0.0398  0.1763   
           random          0.0336  0.0444  0.0380  0.0520  0.0413  0.1768   

                                                      exp_moran          \
n                                                            15           
zones                       2       3       4       5         1       2   
zonal_sort bar_sort                                                       
None       distance_0  0.1814  0.1731  0.1756  0.1865   -0.4038 -0.2652   
           lexico      0.1769  0.1816  0.1683  0.1776   -0.3041 -0.2734   
           max         0.1706  0.1653  0.1774  0.1870   -0.3482 -0.3034   
           projection  0.1750  0.1826  0.1972  0.1768   -0.4052 -0.3066   
           random      0.1682  0.1672  0.1757  0.1827   -0.2846 -0.2478   
distance_0 distance_0  0.1813  0.1768  0.1854  0.1772   -0.3813 -0.3597   
           lexico      0.1739  0.1762  0.1772  0.1814   -0.3013 -0.2847   
           max         0.1787  0.1805  0.1760  0.1785   -0.3254 -0.3255   
           projection  0.1894  0.1808  0.1857  0.1840   -0.3796 -0.3716   
           random      0.1722  0.1778  0.1768  0.1745   -0.2890 -0.3199   
lexico     distance_0  0.1888  0.1638  0.1729  0.1888   -0.3829 -0.3233   
           lexico      0.1709  0.1765  0.1799  0.1738   -0.3069 -0.3397   
           max         0.1809  0.1715  0.1783  0.1699   -0.3236 -0.2938   
           projection  0.1816  0.1739  0.1779  0.1780   -0.3712 -0.3524   
           random      0.1702  0.1825  0.1773  0.1798

In [28]:
top_n_records(df, ['zonal_sort', 'bar_sort', 'zones'], k=10)

,name,exp_moran
0,projection - projection - 5,-0.4134
1,None - projection - 1,-0.4052
2,None - distance_0 - 1,-0.4038
3,distance_0 - max - 5,-0.4029
4,projection - projection - 1,-0.3996
5,random - distance_0 - 1,-0.3915
6,distance_0 - projection - 5,-0.3891
7,projection - lexico - 3,-0.3890
8,projection - distance_0 - 5,-0.3886
9,random - projection - 1,-0.3878


In [29]:
def evaluate_meuse_loop(
        coords_dict,
        probs_dict,
        num_loops=10,
        n_values=[4],
        zone_list=[(1, 1), (2, 2), (3, 3)],
        sort_method_list=['lexico', 'random', 'angle_0', 'distance_0', 'projection', "center", "spiral", "max"],
        zonal_sort_list=['lexico', 'random', 'angle_0', 'distance_0', 'projection', "center", "spiral", "max"],
        zone_mode_list=['sweep' 'cluster'],
        tolerance=5,
        split_size=1e-3,
):
    records = []

    # pre‐compute all combinations
    combos = list(itertools.product(
        n_values,
        zone_list,
        zone_mode_list,
        zonal_sort_list,
        sort_method_list
    ))
    coords = coords_dict['meuse']
    probs = probs_dict['meuse']['equal']

    best_moran_sofar = 0

    for _ in range(num_loops):
        for n, zones, zone_mode, zonal_sort, sort_method in tqdm(
            combos,
            desc="Total combos",
            unit="combo"
        ):

            # print(n, zones, zone_mode, zonal_sort, sort_method)

            modified_probs = inclusion_probabilities(probs, n=n)
            kss = gs.sampling.KMeansSpatialSamplingSimple(
                coords, modified_probs,
                n=n,
                n_zones=zones,
                tolerance=tolerance,
                split_size=split_size,
                zone_mode=zone_mode,
                sort_method=sort_method,
                zonal_sort=zonal_sort,
                max_missed_samples=1
            )

            density_expected = np.round(kss.expected_score(), 4)
            density_val = np.round(kss.var_score(), 4)

            moran_scores, lb_scores = score_all_samples_moran_lb(coords, modified_probs, kss.all_samples)

            moran_expected = np.round(kss.expected_score(moran_scores), 4)
            moran_val = np.round(kss.var_score(moran_scores), 4)

            lb_expected = np.round(kss.expected_score(lb_scores), 4)
            lb_val = np.round(kss.var_score(lb_scores), 4)

            if moran_expected < best_moran_sofar:
                best_moran_sofar = moran_expected
                print('\n====================================')
                print('A NEW BEST FOUND')
                print(f'Moran score: {moran_expected}')
                print(f'zone_mode: {zone_mode}')
                print(f'zonal_sort: {zonal_sort}')
                print(f'bar_sort: {sort_method}')
                print(f'zones: {zones}')

            records.append({
                'n': n,
                'zones': zones if zone_mode == 'cluster' else f"{zones[0]}×{zones[1]}",
                'zone_mode': zone_mode,
                'bar_sort': sort_method,
                'zonal_sort': zonal_sort if zonal_sort else 'None',
                'exp_density': density_expected,
                'exp_moran': moran_expected,
                'exp_lb': lb_expected,
                'var_density': density_val,
                'var_moran': moran_val,
                'var_lb': lb_val,
            })

    return pd.DataFrame.from_records(records)


In [32]:
df = evaluate_meuse_loop(
    coords_dict,
    probs_dict,
    num_loops=50,
    n_values=[15],
    zone_list=[(1, 1), (2, 2), (2, 1), (1, 2)],
    # zone_list=[1, 2, 3, 4, 5],
    sort_method_list=['random'],
    zonal_sort_list=[None, 'random'],
    # zonal_sort_list=[None],
    zone_mode_list=['sweep'],
    tolerance=5,
    split_size=1e-3,
)

Total combos:  12%|█▎        | 1/8 [00:02<00:18,  2.71s/combo]


A NEW BEST FOUND
Moran score: -0.2759
zone_mode: sweep
zonal_sort: None
bar_sort: random


Total combos:  25%|██▌       | 2/8 [00:04<00:12,  2.05s/combo]


A NEW BEST FOUND
Moran score: -0.2855
zone_mode: sweep
zonal_sort: random
bar_sort: random


Total combos:  38%|███▊      | 3/8 [00:06<00:10,  2.08s/combo]

...


A NEW BEST FOUND
Moran score: -0.3725
zone_mode: sweep
zonal_sort: None
bar_sort: random


Total combos:  38%|███▊      | 3/8 [00:06<00:10,  2.11s/combo]


A NEW BEST FOUND
Moran score: -0.3869
zone_mode: sweep
zonal_sort: None
bar_sort: random


Total combos:  88%|████████▊ | 7/8 [00:14<00:02,  2.00s/combo]


KeyboardInterrupt: 

In [34]:
summary = df.pivot_table(
    columns=['sort'],
    values=['exp_density', 'exp_moran', 'var_density', 'var_moran'],
    index=['zones'],
    aggfunc='first'
)

summary

KeyError: 'sort'